In [1]:
# Convolutional - сверточный
# Convolutional neural nets - сверточные нейрнные сети
import torch
import torch.nn as nn # Import NN
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
trans = transforms.Compose([transforms.ToTensor()])

trainset = datasets.MNIST("Datasets", download=True, train=True, transform=trans)
testset = datasets.MNIST("Datasets", download=True, train=False, transform=trans)

batch_train = 8

trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_train)
testloader = DataLoader(testset, shuffle=True)

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
6 * 5 * 5 + 6 * 2 * 2 + 16 * 5 * 5 + 16*2*2 + 256*120 + 120 + 120*84 + 84 + 84*10 + 10

42492

In [4]:
y = [2.0, 1.0, 0.1]
exps = np.exp(y)
y_hat = exps/np.sum(exps)
print(np.sum(y_hat))

1.0


In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # 1 - in_chanel, 6 - out_chanel, 5 - kernel (5x5 filter) 
        self.relu1 = nn.LeakyReLU()
        self.avgp1 = nn.AvgPool2d(2, stride=2)
        self.batchnorm1 = nn.BatchNorm2d(6)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.LeakyReLU()
        self.avgp2 = nn.AvgPool2d(2, stride=2)        
        self.batchnorm2 = nn.BatchNorm2d(16)
        
        self.fc1 = nn.Linear(256, 120)
        self.relu3 = nn.LeakyReLU()
        self.batchnorm3 = nn.BatchNorm2d(120)
        
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.LeakyReLU()
        self.batchnorm4 = nn.BatchNorm2d(84)
        
        
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0) #Sigmoid()
        
    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.avgp1(y)
        y = self.batchnorm1(y)
        
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.avgp2(y)
        y = self.batchnorm2(y)
        
        y = y.reshape(y.shape[0], -1) # convert 5x5x16 to 400x1
        
        y = self.fc1(y)
        y = self.relu3(y)
        y = self.batchnorm3(y)
        
        y = self.fc2(y)
        y = self.relu4(y)
        y = self.batchnorm4(y)
        
        y = self.fc3(y)
        y = self.soft(y)
        return y


model = LeNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device=}")
model.to(device)
print(next(model.parameters()).device)
optimizer = optim.Adam(model.parameters(), lr=1e-3) #optim.SGD(model.parameters(), lr=0.9, momentum=0.9)
criterion = nn.CrossEntropyLoss()

Device: device=device(type='cpu')
cpu


In [6]:
print("Begin train:")
losses = {}
epochs = 1
for epoch in range(epochs):
    model.train()
    for idx, (train_x, train_y) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()

        train_x = train_x.to(device)
        train_y = train_y.to(device)
        predict_y = model(train_x)
        predict_y = predict_y.reshape(batch_train, -1)

        loss = criterion(predict_y, train_y)
        if idx % 100 == 0:
            print(f"Loss: {loss:>7f}")
            losses[idx] = loss
        loss.backward()
        optimizer.step()
print("End of train")

Begin train:


  0%|                                                                                                 | 0/7500 [00:00<?, ?it/s]


ValueError: expected 4D input (got 2D input)

In [ ]:
def show_losses(ls: dict):
    plt.figure(figsize=(12, 8))

    Xs = [ float(x) for x in list(ls.keys())]
    #Xs = Xs[::100]
    Ys = [ float(x) for x in list(ls.values())]
    #Ys = Ys[::100]

    plt.plot(Xs, Ys)
    plt.show()
    
show_losses(losses)
    
total_train = len(trainset)
total_test = len(testset)
with torch.no_grad():
    print("Calculate accuracy on the train set")
    correct = 0
    for image, labels in tqdm(trainloader):
        image, labels = image.to(device), labels.to(device)
        y = model(image)
        y = y.reshape(1, -1)         
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())        
    print(f"Train set accuracy: {(correct/total_train):.4}")
    
    print("Calculate accuracy on the test set")
    correct = 0
    for image, labels in tqdm(testloader):
        image, labels = image.to(device), labels.to(device)
        y = model(image)
        y = y.reshape(1, -1)         
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())        
    print(f"Test set accuracy: {(correct/total_test):.4}")